In [1]:
import pandas as pd
import numpy as np

from ETL import ETL_class

In [10]:
path_titles = './dataset/titles/'
path_rating = './dataset/ratings/'
etl = ETL_class(path_titles, path_rating)

df_movies = etl.get_movies()[['id', 'title', 'cast', 'release_year','duration_int',
                              'duration_type']]
print(df_movies.shape)
df_movies.head(2)

(22998, 6)


,id,title,cast,release_year,duration_int,duration_type
0,as1,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2014,113.0,min
1,as2,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2018,110.0,min


In [3]:
df_rating = etl.get_ratings()
print(df_rating.shape)
df_rating.head()

(11024165, 4)


,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [9]:
mean_rating = df_rating[['movieId', 'rating']].groupby(['movieId']).mean().reset_index()
mean_rating.rename(columns={'rating':'mean_rating', 'movieId': 'id'}, inplace=True)
mean_rating.head()

,id,mean_rating
0,as1,3.467131
1,as10,3.439571
2,as100,3.609302
3,as1000,3.556701
4,as1001,3.585288


In [12]:
merge = pd.merge(df_movies, mean_rating, on='id', how='left')
print(merge.shape)
merge.head()

(22998, 7)


,id,title,cast,release_year,duration_int,duration_type,mean_rating
0,as1,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2014,113.0,min,3.467131
1,as2,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2018,110.0,min,3.548682
2,as3,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2017,74.0,min,3.500000
3,as4,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2014,69.0,min,3.538055
4,as5,monster maker,"harry dean stanton, kieran o'brien, george cos...",1989,45.0,min,3.478992


In [13]:
merge.isna().sum()

id                  0
title               0
cast             5321
release_year        0
duration_int      482
duration_type     482
mean_rating         0
dtype: int64

In [16]:
platform = ['amazon_prime_titles', 'disney_plus_titles', 'hulu_titles', 'netflix_titles']
for p in platform:
    begins = p[0]
    df_aux = merge.loc[merge['id'].str.contains('^{}'.format(begins))]
    df_aux.to_csv('./../first_project/' + p + '.csv', index=False)